In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Subset
from torchvision import datasets
import numpy as np
import matplotlib.pyplot as plt
import torch.nn.functional as F
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score, precision_score, classification_report, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight
import copy
from tqdm import tqdm
import optuna
import warnings
warnings.filterwarnings("ignore")

import sys
sys.path.append('../Utils')
import configs


In [3]:
class DeepCNN(nn.Module):
    def __init__(self, num_classes=7, dropout_prob=0.5):
        super(DeepCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1, 1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, 3, 1, 1)
        self.bn2 = nn.BatchNorm2d(64)
        self.pool1 = nn.MaxPool2d(2,2)
        
        self.conv3 = nn.Conv2d(64,128,3,1,1)
        self.bn3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128,128,3,1,1)
        self.bn4 = nn.BatchNorm2d(128)
        self.pool2 = nn.MaxPool2d(2,2)
        
        self.conv5 = nn.Conv2d(128,256,3,1,1)
        self.bn5 = nn.BatchNorm2d(256)
        self.pool3 = nn.MaxPool2d(2,2)
        
        self.dropout = nn.Dropout(dropout_prob)
        self.fc1 = nn.Linear(256*28*28, 512)
        self.fc2 = nn.Linear(512,256)
        self.fc3 = nn.Linear(256,num_classes)
        
        self._initialize_weights()
        
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
        
    def forward(self,x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.pool1(x)
        
        x = F.relu(self.bn3(self.conv3(x)))
        x = F.relu(self.bn4(self.conv4(x)))
        x = self.pool2(x)
        
        x = F.relu(self.bn5(self.conv5(x)))
        x = self.pool3(x)
        
        x = x.view(x.size(0),-1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x


In [4]:
# Training augmentation
transform_train = transforms.Compose([
    transforms.Grayscale(1),
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(0.2),
    transforms.RandomRotation(15),
    transforms.ColorJitter(0.2,0.2,0.1),
    transforms.RandomAffine(0, translate=(0.1,0.1), scale=(0.9,1.1)),
    transforms.RandomPerspective(0.1,0.2),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Validation / Test (no augmentation)
transform_test = transforms.Compose([
    transforms.Grayscale(1),
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Datasets
train_dataset = datasets.ImageFolder(configs.MULTIVIEW_TRAIN_DIR, transform=transform_train)
test_dataset = datasets.ImageFolder(configs.MULTIVIEW_TEST_DIR, transform=transform_test)

# Train / Validation Split
train_size = int(0.8*len(train_dataset))
val_size = len(train_dataset)-train_size
train_subset, val_subset = torch.utils.data.random_split(train_dataset,[train_size,val_size])

val_dataset_no_aug = datasets.ImageFolder(configs.MULTIVIEW_TRAIN_DIR, transform=transform_test)
val_subset_no_aug = Subset(val_dataset_no_aug, val_subset.indices)

# DataLoaders
train_loader = DataLoader(train_subset, batch_size=16, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_subset_no_aug, batch_size=16, shuffle=False, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=4, pin_memory=True)


In [5]:
def train_one_epoch(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss, running_corrects, total = 0, 0, 0
    for inputs, labels in dataloader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        _, preds = torch.max(outputs,1)
        loss = criterion(outputs,labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()*inputs.size(0)
        running_corrects += torch.sum(preds==labels.data)
        total += inputs.size(0)
    return running_loss/total, running_corrects.double()/total

def validate(model, dataloader, criterion, device):
    model.eval()
    running_loss, running_corrects, total = 0,0,0
    all_preds, all_labels = [], []
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs,1)
            loss = criterion(outputs,labels)
            running_loss += loss.item()*inputs.size(0)
            running_corrects += torch.sum(preds==labels.data)
            total += inputs.size(0)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    return running_loss/total, running_corrects.double()/total, all_labels, all_preds

def train_model_with_history(model, train_loader, val_loader, criterion, optimizer, scheduler=None, 
                             num_epochs=50, patience=10, device=None):
    history = {"train_loss":[],"train_acc":[],"val_loss":[],"val_acc":[]}
    best_acc = 0
    best_model_wts = copy.deepcopy(model.state_dict())
    patience_counter = 0
    if device is None:
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    for epoch in range(num_epochs):
        train_loss, train_acc = train_one_epoch(model, train_loader, criterion, optimizer, device)
        val_loss, val_acc, _, _ = validate(model, val_loader, criterion, device)
        
        history["train_loss"].append(train_loss)
        history["train_acc"].append(train_acc.item())
        history["val_loss"].append(val_loss)
        history["val_acc"].append(val_acc.item())
        
        if scheduler:
            scheduler.step(val_loss)
            
        if val_acc>best_acc:
            best_acc = val_acc
            best_model_wts = copy.deepcopy(model.state_dict())
            patience_counter=0
        else:
            patience_counter+=1
            if patience_counter>=patience:
                print(f"Early stopping at epoch {epoch+1}")
                break
        
        print(f"Epoch {epoch+1}: Train Loss={train_loss:.4f}, Train Acc={train_acc:.4f}, "
              f"Val Loss={val_loss:.4f}, Val Acc={val_acc:.4f}")
    model.load_state_dict(best_model_wts)
    return model, history


In [6]:
def evaluate_model(model, dataloader, device=None):
    if device is None:
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs,1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    acc = accuracy_score(all_labels, all_preds)
    bal_acc = balanced_accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average='weighted')
    prec = precision_score(all_labels, all_preds, average='weighted')
    print(f"Accuracy={acc:.4f}, Balanced Acc={bal_acc:.4f}, F1={f1:.4f}, Precision={prec:.4f}")
    print("\nClassification Report:\n", classification_report(all_labels, all_preds))
    return all_labels, all_preds


In [7]:
def plot_learning_curves(history):
    plt.figure(figsize=(12,5))
    plt.subplot(1,2,1)
    plt.plot(history["train_loss"], label="Train Loss")
    plt.plot(history["val_loss"], label="Val Loss")
    plt.xlabel("Epochs"); plt.ylabel("Loss"); plt.title("Loss Curve"); plt.legend(); plt.grid(True, alpha=0.3)
    
    plt.subplot(1,2,2)
    plt.plot(history["train_acc"], label="Train Acc")
    plt.plot(history["val_acc"], label="Val Acc")
    plt.xlabel("Epochs"); plt.ylabel("Accuracy"); plt.title("Accuracy Curve"); plt.legend(); plt.grid(True, alpha=0.3)
    
    plt.show()


In [ ]:
def objective(trial):
    # Hyperparameter search space
    lr = trial.suggest_float("lr", 1e-4, 1e-2, log=True)
    dropout = trial.suggest_float("dropout", 0.3, 0.6)
    weight_decay = trial.suggest_float("weight_decay", 1e-5, 1e-3, log=True)

    # Subsample training data for speed (30% of original)
    subset_size = int(0.3 * len(train_subset))
    subset_indices = np.random.choice(train_subset.indices, subset_size, replace=False)
    trial_subset = torch.utils.data.Subset(train_dataset, subset_indices)
    trial_loader = DataLoader(trial_subset, batch_size=16, shuffle=True, num_workers=2)
    
    model = DeepCNN(num_classes=7, dropout_prob=dropout)
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    # Compute class weights on subset
    trial_labels = [train_dataset.targets[i] for i in subset_indices]
    classes = np.unique(trial_labels)
    class_weights = compute_class_weight('balanced', classes=classes, y=trial_labels)
    class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)

    criterion = nn.CrossEntropyLoss(weight=class_weights)
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=2, factor=0.5)

    # Train for fewer epochs just to evaluate hyperparameters quickly
    model, history = train_model_with_history(
        model, trial_loader, val_loader,
        criterion, optimizer, scheduler,
        num_epochs=8,  # much shorter
        device=device,
        patience=3     # very short patience to exit bad trials early
    )
    
    # Use best validation accuracy achieved
    return max(history["val_acc"])

# Run Optuna study
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10, show_progress_bar=True)  # fewer trials for faster search

print("Best trial parameters:", study.best_trial.params)
print("Best validation accuracy:", study.best_value)

[I 2025-09-11 12:35:35,382] A new study created in memory with name: no-name-dadf5ed7-f40d-424d-b1b7-dd516bfaf2ef


  0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
# %% [markdown]
# # DGCNN: Direct 3D Point Cloud Classification
# - FPS sampling
# - Augmentation
# - Training, Evaluation, Metrics, and Visualization

# %% Imports & Setup
import os, math, time, random
from types import SimpleNamespace
from pathlib import Path
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import sys
sys.path.append('../Utils')  # adjust if your models/configs are elsewhere
import configs
from models import DGCNN

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.backends.cudnn.benchmark = True
print("Device:", device)

# Directories
TRAIN_SET = configs.TRAIN_DIR
TEST_SET = configs.TEST_DIR
MODELS_SAVE_DIR = configs.MODEL_DIR

# Config
CFG = SimpleNamespace(
    num_points=1024,
    k=20,
    emb_dims=1024,
    dropout=0.5,
    batch_size=16,
    epochs=80,
    patience=12,
    base_lr=1e-3,
    min_lr=1e-5,
    weight_decay=1e-4,
    label_smoothing=0.1,
    num_workers=4,
    seed=42
)

# Seed
def set_seed(s=42):
    random.seed(s); np.random.seed(s)
    torch.manual_seed(s); torch.cuda.manual_seed_all(s)
set_seed(CFG.seed)


Device: cpu


In [ ]:

# # FPS Sampling, Normalization, and Augmentations

# %%
def farthest_point_sampling(points: np.ndarray, n_samples: int) -> np.ndarray:
    N = points.shape[0]
    if N == 0:
        return np.zeros((n_samples, 3), dtype=np.float32)
    n_samples = min(n_samples, N)
    centroids = np.zeros((n_samples,), dtype=np.int64)
    distances = np.ones((N,), dtype=np.float32) * 1e10
    farthest = np.random.randint(0, N)
    for i in range(n_samples):
        centroids[i] = farthest
        centroid = points[farthest]
        dist = np.sum((points - centroid) ** 2, axis=1)
        sel = dist < distances
        distances[sel] = dist[sel]
        farthest = np.argmax(distances)
    sampled = points[centroids]
    if sampled.shape[0] < CFG.num_points:
        pad_idx = np.random.choice(len(sampled), CFG.num_points - len(sampled), replace=True)
        sampled = np.concatenate([sampled, sampled[pad_idx]], axis=0)
    return sampled.astype(np.float32)

def normalize_unit(points: np.ndarray) -> np.ndarray:
    c = points.mean(axis=0, keepdims=True)
    p = points - c
    scale = np.max(np.sqrt((p**2).sum(1))) + 1e-8
    return (p / scale).astype(np.float32)

def random_jitter(points: np.ndarray, sigma=0.01, clip=0.05) -> np.ndarray:
    noise = np.clip(sigma * np.random.randn(*points.shape), -clip, clip).astype(np.float32)
    return (points + noise).astype(np.float32)

def random_scale(points: np.ndarray, low=0.85, high=1.2) -> np.ndarray:
    s = np.random.uniform(low, high)
    return (points * s).astype(np.float32)

def random_rotation_z(points: np.ndarray) -> np.ndarray:
    theta = np.random.uniform(0, 2*np.pi)
    c, s = np.cos(theta), np.sin(theta)
    R = np.array([[c, -s, 0], [s, c, 0], [0, 0, 1]], dtype=np.float32)
    return (points @ R.T).astype(np.float32)

def load_txt_points(path: Path) -> np.ndarray:
    pts = np.loadtxt(path).astype(np.float32)
    if pts.ndim == 1: pts = pts[None, :]
    if pts.shape[1] > 3: pts = pts[:, :3]
    return pts

def list_point_files(root: Path):
    files = []
    for ext in ("*.pts", "*.xyz", "*.txt"):
        files.extend(sorted((root).rglob(ext)))
    return files


In [ ]:
# %% [markdown]
# # Dataset & DataLoader

# %%
class PointCloudDataset(Dataset):
    def __init__(self, root_dir: Path, num_points=1024, augment=False):
        self.root_dir = Path(root_dir)
        self.num_points = num_points
        self.augment = augment
        self.classes = sorted([d.name for d in self.root_dir.iterdir() if d.is_dir()])
        self.class_to_idx = {c:i for i,c in enumerate(self.classes)}
        self.samples = []
        for cls in self.classes:
            for f in list_point_files(self.root_dir/cls):
                self.samples.append((f, self.class_to_idx[cls]))
        if len(self.samples) == 0:
            raise RuntimeError(f"No point files found under {root_dir}")

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        path, label = self.samples[idx]
        pts = load_txt_points(path)
        pts = farthest_point_sampling(pts, self.num_points)
        pts = normalize_unit(pts)
        if self.augment:
            pts = random_rotation_z(pts)
            pts = random_scale(pts)
            pts = random_jitter(pts)
        return torch.from_numpy(pts.T), torch.tensor(label, dtype=torch.long)

# Create Datasets and Loaders
train_ds = PointCloudDataset(TRAIN_SET, num_points=CFG.num_points, augment=True)
test_ds  = PointCloudDataset(TEST_SET,  num_points=CFG.num_points, augment=False)

num_classes = len(train_ds.classes)
print(f"Classes ({num_classes}): {train_ds.classes}")

train_loader = DataLoader(train_ds, batch_size=CFG.batch_size, shuffle=True,
                          num_workers=CFG.num_workers, pin_memory=True)
test_loader  = DataLoader(test_ds, batch_size=CFG.batch_size, shuffle=False,
                          num_workers=CFG.num_workers, pin_memory=True)


Classes (7): ['Ash', 'Beech', 'Douglas Fir', 'Oak', 'Pine', 'Red Oak', 'Spruce']


In [ ]:
# %% [markdown]
# # Model, Optimizer, Scheduler, Loss

# %%
# Model
model = DGCNN(num_classes=num_classes, k=CFG.k, emb_dims=CFG.emb_dims, dropout=CFG.dropout)
model = model.to(device)

# Loss with label smoothing
criterion = nn.CrossEntropyLoss(label_smoothing=CFG.label_smoothing)

# Optimizer (AdamW for stability)
optimizer = torch.optim.AdamW(model.parameters(), lr=CFG.base_lr, weight_decay=CFG.weight_decay)

# Cosine Annealing LR Scheduler
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer, T_max=CFG.epochs, eta_min=CFG.min_lr
)

print(f"Model Parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")
print(f"Optimizer: AdamW | Scheduler: CosineAnnealingLR")


TypeError: DGCNN.__init__() got an unexpected keyword argument 'num_classes'

In [ ]:
# %% [markdown]
# # Training and Evaluation Functions

# %%
def train_one_epoch(model, loader, optimizer, criterion):
    model.train()
    total_loss, total_correct, total_samples = 0, 0, 0
    for points, labels in loader:
        points, labels = points.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(points)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        preds = outputs.argmax(dim=1)
        total_loss += loss.item() * labels.size(0)
        total_correct += (preds == labels).sum().item()
        total_samples += labels.size(0)
    return total_loss / total_samples, total_correct / total_samples


@torch.no_grad()
def evaluate(model, loader, criterion):
    model.eval()
    total_loss, total_correct, total_samples = 0, 0, 0
    all_labels, all_preds = [], []
    for points, labels in loader:
        points, labels = points.to(device), labels.to(device)
        outputs = model(points)
        loss = criterion(outputs, labels)
        preds = outputs.argmax(dim=1)

        total_loss += loss.item() * labels.size(0)
        total_correct += (preds == labels).sum().item()
        total_samples += labels.size(0)

        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(preds.cpu().numpy())

    return (
        total_loss / total_samples,
        total_correct / total_samples,
        np.array(all_labels),
        np.array(all_preds)
    )


In [ ]:
# %% [markdown]
# # Training Loop with Early Stopping and History Tracking

# %%
train_losses, val_losses, train_accs, val_accs = [], [], [], []
best_val_acc = 0.0
patience_counter = 0
best_model_wts = None

for epoch in range(CFG.epochs):
    start_time = time.time()

    train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, criterion)
    val_loss, val_acc, val_labels, val_preds = evaluate(model, test_loader, criterion)

    train_losses.append(train_loss)
    val_losses.append(val_loss)
    train_accs.append(train_acc)
    val_accs.append(val_acc)

    scheduler.step()

    elapsed = time.time() - start_time
    print(f"[Epoch {epoch+1:02d}/{CFG.epochs}] "
          f"Train Loss: {train_loss:.4f} Acc: {train_acc:.4f} | "
          f"Val Loss: {val_loss:.4f} Acc: {val_acc:.4f} | "
          f"Time: {elapsed:.1f}s")

    # Early stopping
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_model_wts = model.state_dict()
        patience_counter = 0
        torch.save(best_model_wts, MODELS_SAVE_DIR / "best_dgcnn.pth")
    else:
        patience_counter += 1
        if patience_counter >= CFG.patience:
            print("Early stopping triggered.")
            break

# Load best model
if best_model_wts is not None:
    model.load_state_dict(best_model_wts)
print(f"Best Val Accuracy: {best_val_acc:.4f}")


In [ ]:
# %% [markdown]
# # Final Evaluation and Classification Metrics

# %%
val_loss, val_acc, val_labels, val_preds = evaluate(model, test_loader, criterion)
print(f"Final Test Loss: {val_loss:.4f} | Final Test Accuracy: {val_acc:.4f}")

print("\nClassification Report:")
print(classification_report(val_labels, val_preds, target_names=train_ds.classes))

cm = confusion_matrix(val_labels, val_preds)
plt.figure(figsize=(6,6))
plt.imshow(cm, cmap="Blues")
plt.title("Confusion Matrix")
plt.colorbar()
plt.xticks(np.arange(num_classes), train_ds.classes, rotation=45)
plt.yticks(np.arange(num_classes), train_ds.classes)
plt.xlabel("Predicted")
plt.ylabel("True")
plt.show()


In [ ]:
# %% [markdown]
# # Training History Visualization

# %%
epochs_ran = len(train_losses)
plt.figure(figsize=(12,4))

plt.subplot(1,2,1)
plt.plot(range(1, epochs_ran+1), train_losses, label='Train Loss')
plt.plot(range(1, epochs_ran+1), val_losses, label='Val Loss')
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training & Validation Loss")
plt.legend()

plt.subplot(1,2,2)
plt.plot(range(1, epochs_ran+1), train_accs, label='Train Acc')
plt.plot(range(1, epochs_ran+1), val_accs, label='Val Acc')
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.title("Training & Validation Accuracy")
plt.legend()

plt.show()


In [ ]:
best_params = study.best_trial.params
model = DeepCNN(num_classes=7, dropout_prob=best_params["dropout"])
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

train_labels = [train_dataset.targets[i] for i in train_subset.indices]
classes = np.unique(train_labels)
class_weights = compute_class_weight('balanced', classes=classes, y=train_labels)
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)

criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.Adam(model.parameters(), lr=best_params["lr"], weight_decay=best_params["weight_decay"])
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=3, factor=0.5)

model, history = train_model_with_history(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=50, patience=10, device=device)
plot_learning_curves(history)


In [ ]:
all_labels, all_preds = evaluate_model(model, test_loader, device)
